In [1]:
#needed libraries
import os
import gzip
import json
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')

In [2]:
"""
import tarfile
with tarfile.open("flaws_cloudtrail_logs.tar", "r") as tar:
    for member in tar.getmembers():
        print(member.name)

"""

'\nimport tarfile\nwith tarfile.open("flaws_cloudtrail_logs.tar", "r") as tar:\n    for member in tar.getmembers():\n        print(member.name)\n\n'

In [3]:


log_dir = "extracted/flaws_cloudtrail_logs"

all_records = []

for file in os.listdir(log_dir):
    if file.endswith(".json.gz"):
        file_path = os.path.join(log_dir, file)

        with gzip.open(file_path, "rt") as f:
            data = json.load(f)        # Load the entire JSON object
            records = data.get("Records", [])
            all_records.extend(records)  

In [4]:
df = pd.DataFrame(all_records)
df.head()

,userAgent,eventID,userIdentity,eventType,sourceIPAddress,eventName,eventSource,recipientAccountId,requestParameters,awsRegion,...,errorCode,apiVersion,readOnly,resources,additionalEventData,sharedEventID,vpcEndpointId,managementEvent,eventCategory,serviceEventDetails
0,[S3Console/0.4],3038ebd2-c98a-4c65-9b6e-e22506292313,"{'type': 'Root', 'principalId': '811596193553'...",AwsApiCall,255.253.125.115,ListBuckets,s3.amazonaws.com,811596193553,None,us-east-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,console.amazonaws.com,22a0d9b1-deea-4d39-827b-2af7050ed3f3,"{'type': 'Root', 'principalId': '811596193553'...",AwsApiCall,255.253.125.115,GetAccountPasswordPolicy,iam.amazonaws.com,811596193553,None,us-east-1,...,NoSuchEntityException,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,console.amazonaws.com,9facf7ca-cb76-4b19-940c-3de6803f7efb,"{'type': 'Root', 'principalId': '811596193553'...",AwsApiCall,255.253.125.115,GetAccountSummary,iam.amazonaws.com,811596193553,None,us-east-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,console.amazonaws.com,6596d3b4-7c98-40b1-867d-f317f1dbdc18,"{'type': 'Root', 'principalId': '811596193553'...",AwsApiCall,255.253.125.115,ListAccountAliases,iam.amazonaws.com,811596193553,None,us-east-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,console.amazonaws.com,9f9d038c-e5a5-443e-83d5-4cf00941d399,"{'type': 'Root', 'principalId': '811596193553'...",AwsApiCall,255.253.125.115,ListMFADevices,iam.amazonaws.com,811596193553,None,us-east-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Data Preprocessing**

In [5]:
df.isna().sum()

userAgent                    1
eventID                      0
userIdentity                 0
eventType                    0
sourceIPAddress              0
eventName                    0
eventSource                  0
recipientAccountId           0
requestParameters       176327
awsRegion                    0
requestID                  703
responseElements       1849020
eventVersion                 0
eventTime                    0
errorMessage            439435
errorCode               431493
apiVersion             1923344
readOnly               1885894
resources              1870935
additionalEventData    1875152
sharedEventID          1878944
vpcEndpointId          1938490
managementEvent        1935585
eventCategory          1938764
serviceEventDetails    1938604
dtype: int64

In [6]:
cols_to_drop = ['eventID','requestID', 'eventVersion', 'recipientAccountId','responseElements','apiVersion','serviceEventDetails','eventCategory','managementEvent',
                'vpcEndpointId','sharedEventID','additionalEventData','resources']

df = df.drop(columns=cols_to_drop)


In [7]:
df.isna().sum()

userAgent                  1
userIdentity               0
eventType                  0
sourceIPAddress            0
eventName                  0
eventSource                0
requestParameters     176327
awsRegion                  0
eventTime                  0
errorMessage          439435
errorCode             431493
readOnly             1885894
dtype: int64

In [8]:
df

,userAgent,userIdentity,eventType,sourceIPAddress,eventName,eventSource,requestParameters,awsRegion,eventTime,errorMessage,errorCode,readOnly
0,[S3Console/0.4],"{'type': 'Root', 'principalId': '811596193553'...",AwsApiCall,255.253.125.115,ListBuckets,s3.amazonaws.com,None,us-east-1,2017-02-12T19:57:06Z,NaN,NaN,NaN
1,console.amazonaws.com,"{'type': 'Root', 'principalId': '811596193553'...",AwsApiCall,255.253.125.115,GetAccountPasswordPolicy,iam.amazonaws.com,None,us-east-1,2017-02-12T19:59:10Z,The Password Policy with domain name 811596193...,NoSuchEntityException,NaN
2,console.amazonaws.com,"{'type': 'Root', 'principalId': '811596193553'...",AwsApiCall,255.253.125.115,GetAccountSummary,iam.amazonaws.com,None,us-east-1,2017-02-12T19:59:10Z,NaN,NaN,NaN
3,console.amazonaws.com,"{'type': 'Root', 'principalId': '811596193553'...",AwsApiCall,255.253.125.115,ListAccountAliases,iam.amazonaws.com,None,us-east-1,2017-02-12T19:59:10Z,NaN,NaN,NaN
4,console.amazonaws.com,"{'type': 'Root', 'principalId': '811596193553'...",AwsApiCall,255.253.125.115,ListMFADevices,iam.amazonaws.com,None,us-east-1,2017-02-12T19:59:10Z,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1939202,[Boto3/1.15.13 Python/3.8.5 Darwin/19.6.0 Boto...,"{'type': 'IAMUser', 'principalId': 'AIDA9BO36H...",AwsApiCall,193.29.252.218,GetBucketAcl,s3.amazonaws.com,{'bucketName': 'provision-governance.appgyver....,us-west-2,2020-10-07T21:03:27Z,The specified bucket does not exist,NoSuchBucket,True
1939203,[Boto3/1.15.13 Python/3.8.5 Darwin/19.6.0 Boto...,"{'type': 'IAMUser', 'principalId': 'AIDA9BO36H...",AwsApiCall,193.29.252.218,GetBucketAcl,s3.amazonaws.com,{'bucketName': 'provision-guides.appgyver.acad...,us-west-2,2020-10-07T21:03:27Z,The specified bucket does not exist,NoSuchBucket,True
1939204,[Boto3/1.15.13 Python/3.8.5 Darwin/19.6.0 Boto...,"{'type': 'IAMUser', 'principalId': 'AIDA9BO36H...",AwsApiCall,193.29.252.218,GetBucketAcl,s3.amazonaws.com,{'bucketName': 'provision-hack.appgyver.academ...,us-west-2,2020-10-07T21:03:28Z,The specified bucket does not exist,NoSuchBucket,True
1939205,[Boto3/1.15.13 Python/3.8.5 Darwin/19.6.0 Boto...,"{'type': 'IAMUser', 'principalId': 'AIDA9BO36H...",AwsApiCall,193.29.252.218,GetBucketAcl,s3.amazonaws.com,{'bucketName': 'provision-inbound.appgyver.aca...,us-west-2,2020-10-07T21:03:29Z,The specified bucket does not exist,NoSuchBucket,True


In [9]:
# fill missing user agent values
df['userAgent'] = df['userAgent'].fillna('Unknown')

# fill missing values
df['requestParameters'] = df['requestParameters'].fillna("{}")



**Feature Engineering**

In [10]:
# create isWriteOperation from readOnly
df['isWriteOperation'] = df['readOnly'].apply(lambda x: 1 if x == False else 0)

# drop readOnly original column
df = df.drop('readOnly', axis=1)

# fill missing error fields
df[['errorMessage','errorCode']] = df[['errorMessage','errorCode']].fillna('None')

# create binary error flag
df['hasError'] = (df['errorCode'] != 'None').astype(int)

In [11]:

# flag if parameters exist
df['hasRequestParams'] = (df['requestParameters'] != "{}").astype(int)

# count number of keys
def count_keys(x):
    try:
        return len(json.loads(x))
    except:
        return 0
df['paramNumKeys'] = df['requestParameters'].apply(count_keys)

# length of the parameter payload
df['paramLength'] = df['requestParameters'].astype(str).apply(len)

In [12]:
df.columns

Index(['userAgent', 'userIdentity', 'eventType', 'sourceIPAddress',
       'eventName', 'eventSource', 'requestParameters', 'awsRegion',
       'eventTime', 'errorMessage', 'errorCode', 'isWriteOperation',
       'hasError', 'hasRequestParams', 'paramNumKeys', 'paramLength'],
      dtype='object')

In [13]:
# semantic threat features
df['hasRole'] = df['requestParameters'].str.contains('role', case=False, na=False).astype(int)
df['hasPolicy'] = df['requestParameters'].str.contains('policy', case=False, na=False).astype(int)

df['hasAccessKey'] = df['requestParameters'].str.contains('accessKey', case=False, na=False).astype(int)
df['hasUser'] = df['requestParameters'].str.contains('userName', case=False, na=False).astype(int)
df['hasBucket'] = df['requestParameters'].str.contains('bucket', case=False, na=False).astype(int)

df['hasSecurityGroup'] = df['requestParameters'].str.contains('securityGroup', case=False, na=False).astype(int)
df['hasInstance'] = df['requestParameters'].str.contains('instanceId', case=False, na=False).astype(int)

In [14]:
# extract the users from the user identity column
def extract_user(x):
    try:
        if isinstance(x, dict):
            return x.get("arn") or x.get("userName") or x.get("principalId")
        else:
            return str(x)
    except:
        return "unknownUser"
    
df['userIdentitySimple'] = df['userIdentity'].apply(extract_user)

In [15]:
# time features
df['eventTime'] = pd.to_datetime(df['eventTime'])

df['hour'] = df['eventTime'].dt.hour
df['day_of_week'] = df['eventTime'].dt.dayofweek
df['isWeekend'] = df['day_of_week'].isin([5,6]).astype(int)

# Time since last event per user
df = df.sort_values(by=['userIdentitySimple', 'eventTime'])
df['timeSinceLastEvent'] = df.groupby('userIdentitySimple')['eventTime'].diff().dt.total_seconds()
df['timeSinceLastEvent'] = df['timeSinceLastEvent'].fillna(df['timeSinceLastEvent'].median())

# Events per user in last hour
df['eventHour'] = df['eventTime'].dt.floor('H')
df['eventsLastHour'] = df.groupby(['userIdentitySimple', 'eventHour'])['eventTime'].transform('count')

# Night-time flag
df['isNight'] = df['hour'].between(0, 6).astype(int)

In [16]:
# drop the unnecessary columns after generating the features
df = df.drop(columns=['userIdentity','requestParameters', 'eventTime'])

In [17]:
# Create semantic flags from errorCode
df['isAccessDenied'] = (df['errorCode'] == 'AccessDenied').astype(int)
df['isNoSuchBucket'] = (df['errorCode'] == 'NoSuchBucket').astype(int)
df['isUnauthorized'] = df['errorCode'].str.contains('Unauthorized', na=False).astype(int)

df = df.drop(columns=['errorMessage','errorCode'])


In [18]:
df

,userAgent,eventType,sourceIPAddress,eventName,eventSource,awsRegion,isWriteOperation,hasError,hasRequestParams,paramNumKeys,...,hour,day_of_week,isWeekend,timeSinceLastEvent,eventHour,eventsLastHour,isNight,isAccessDenied,isNoSuchBucket,isUnauthorized
1880040,[Boto3/1.4.7 Python/2.7.16 Linux/4.15.0-66-gen...,AwsApiCall,254.9.176.211,GetBucketPolicy,s3.amazonaws.com,us-west-2,0,1,1,0,...,2,5,1,0.0,2020-08-15 02:00:00+00:00,16.0,1,1,0,0
1880041,[Boto3/1.4.7 Python/2.7.16 Linux/4.15.0-66-gen...,AwsApiCall,254.9.176.211,GetBucketAcl,s3.amazonaws.com,us-west-2,0,1,1,0,...,2,5,1,0.0,2020-08-15 02:00:00+00:00,16.0,1,1,0,0
1880042,[Boto3/1.4.7 Python/2.7.16 Linux/4.15.0-66-gen...,AwsApiCall,254.9.176.211,GetBucketCors,s3.amazonaws.com,us-west-2,0,1,1,0,...,2,5,1,0.0,2020-08-15 02:00:00+00:00,16.0,1,1,0,0
1880043,[Boto3/1.4.7 Python/2.7.16 Linux/4.15.0-66-gen...,AwsApiCall,254.9.176.211,GetBucketLifecycle,s3.amazonaws.com,us-west-2,0,1,1,0,...,2,5,1,0.0,2020-08-15 02:00:00+00:00,16.0,1,1,0,0
1880044,[Boto3/1.4.7 Python/2.7.16 Linux/4.15.0-66-gen...,AwsApiCall,254.9.176.211,GetBucketNotification,s3.amazonaws.com,us-west-2,0,1,1,0,...,2,5,1,0.0,2020-08-15 02:00:00+00:00,16.0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1933986,ec2.amazonaws.com,AwsApiCall,ec2.amazonaws.com,AssumeRole,sts.amazonaws.com,us-west-2,0,0,1,0,...,19,2,0,0.0,2020-10-07 19:00:00+00:00,NaN,0,0,0,0
1933987,ec2.amazonaws.com,AwsApiCall,ec2.amazonaws.com,AssumeRole,sts.amazonaws.com,us-west-2,0,0,1,0,...,19,2,0,0.0,2020-10-07 19:00:00+00:00,NaN,0,0,0,0
1935496,config.amazonaws.com,AwsApiCall,config.amazonaws.com,AssumeRole,sts.amazonaws.com,us-west-2,0,0,1,0,...,20,2,0,0.0,2020-10-07 20:00:00+00:00,NaN,0,0,0,0
1938688,ec2.amazonaws.com,AwsApiCall,ec2.amazonaws.com,AssumeRole,sts.amazonaws.com,us-west-2,0,0,1,0,...,20,2,0,0.0,2020-10-07 20:00:00+00:00,NaN,0,0,0,0


**Encoding Categorical Features**

In [19]:
# get dummies for the event type column
df = pd.get_dummies(df, columns=['eventType'], drop_first=True)

#freguency encoding for the high cardinality features
def frequency_encoding(df, col):
    freq = df[col].value_counts(normalize=True)
    df[col] = df[col].fillna("Unknown")
    df[col + '_freq'] = df[col].map(freq)
    df = df.drop(columns=[col])
    return df

df = frequency_encoding(df, 'userAgent')
df = frequency_encoding(df, 'userIdentitySimple')
df = frequency_encoding(df, 'eventSource')
df = frequency_encoding(df, 'awsRegion')
df = frequency_encoding(df, 'eventName')
df = frequency_encoding(df,'sourceIPAddress')


In [20]:
# convert the boolean columns to numeric
bool_cols = df.select_dtypes(include=['bool']).columns
df[bool_cols] = df[bool_cols].astype(int)

#fill thenulls after the encoding
freq_cols = [col for col in df.columns if col.endswith('_freq')]
df[freq_cols] = df[freq_cols].fillna(0)

In [21]:
df['eventHour'] = df['eventHour'].dt.hour

In [22]:
df

,isWriteOperation,hasError,hasRequestParams,paramNumKeys,paramLength,hasRole,hasPolicy,hasAccessKey,hasUser,hasBucket,...,isUnauthorized,eventType_AwsConsoleAction,eventType_AwsConsoleSignIn,eventType_AwsServiceEvent,userAgent_freq,userIdentitySimple_freq,eventSource_freq,awsRegion_freq,eventName_freq,sourceIPAddress_freq
1880040,0,1,1,0,81,0,0,0,0,0,...,0,0,0,0,0.000462,0.000598,0.045375,0.169622,0.001501,0.000462
1880041,0,1,1,0,78,0,0,0,0,0,...,0,0,0,0,0.000462,0.000598,0.045375,0.169622,0.021994,0.000462
1880042,0,1,1,0,79,0,0,0,0,0,...,0,0,0,0,0.000462,0.000598,0.045375,0.169622,0.000829,0.000462
1880043,0,1,1,0,84,0,0,0,0,0,...,0,0,0,0,0.000462,0.000598,0.045375,0.169622,0.001307,0.000462
1880044,0,1,1,0,87,0,0,0,0,0,...,0,0,0,0,0.000462,0.000598,0.045375,0.169622,0.001308,0.000462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1933986,0,0,1,0,93,0,0,0,0,0,...,0,0,0,0,0.022771,0.000000,0.049915,0.169622,0.040904,0.022771
1933987,0,0,1,0,93,0,0,0,0,0,...,0,0,0,0,0.022771,0.000000,0.049915,0.169622,0.040904,0.022771
1935496,0,0,1,0,112,0,0,0,0,0,...,0,0,0,0,0.004039,0.000000,0.049915,0.169622,0.040904,0.004039
1938688,0,0,1,0,93,0,0,0,0,0,...,0,0,0,0,0.022771,0.000000,0.049915,0.169622,0.040904,0.022771


In [23]:
df.columns

Index(['isWriteOperation', 'hasError', 'hasRequestParams', 'paramNumKeys',
       'paramLength', 'hasRole', 'hasPolicy', 'hasAccessKey', 'hasUser',
       'hasBucket', 'hasSecurityGroup', 'hasInstance', 'hour', 'day_of_week',
       'isWeekend', 'timeSinceLastEvent', 'eventHour', 'eventsLastHour',
       'isNight', 'isAccessDenied', 'isNoSuchBucket', 'isUnauthorized',
       'eventType_AwsConsoleAction', 'eventType_AwsConsoleSignIn',
       'eventType_AwsServiceEvent', 'userAgent_freq',
       'userIdentitySimple_freq', 'eventSource_freq', 'awsRegion_freq',
       'eventName_freq', 'sourceIPAddress_freq'],
      dtype='object')

In [24]:
# scale features

X = df.copy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

**Modelling**

In [25]:
# isolation forest
iso = IsolationForest(n_estimators=300, contamination=0.01, random_state=42, n_jobs=-1)

iso.fit(X_scaled)
iso_scores = iso.decision_function(X_scaled)
iso_pred = iso.predict(X_scaled)

In [26]:
# convert predictions
df['iso_anomaly'] = (iso_pred == -1).astype(int)